In [1]:
import os
import sys
sys.path.append('../')
os.chdir('../')

In [2]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np
import duck_db_helper
import duckdb
from datetime import date

from statsmodels.iolib.smpickle import load_pickle

In [4]:
model_name = 'model1'
new_results = load_pickle(f"../data/models/{model_name}.pickle")

table_name = 'test_feature_1'
db = '../data/feature_generation/test.db'
conn = duckdb.connect(db)
test_df = conn.sql(f"SELECT * FROM \"{table_name}\";").df()
conn.close()

In [5]:
test_df['total_pred'] = new_results.predict(test_df)

In [7]:
rss = np.power(test_df['total_points']-test_df['total_pred'], 2).sum()
tss = np.power(test_df['total_points']-test_df['total_points'].mean(), 2).sum()
rsquared = 1 - rss/tss

with open('../data/models/test_registry.csv', mode='a') as f:
    f.write(f'{str(date.today())},{model_name},{db}:{table_name},{rsquared}\n')